In [147]:
#path
import os
from os.path import isdir, join
from pathlib import Path

# Scientific Math 
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.offline as py
import plotly.graph_objs as go

#Deep learning
from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten
from keras import Input, layers
from keras import backend as K

import random
import copy
import librosa
import sounddevice as sd

%matplotlib inline

In [148]:
print(os.listdir("../data"))


['.DS_Store', 'test', 'train']


In [149]:
train_audio_path = '../data/train/audio/'
print(os.listdir(train_audio_path))

['right', 'eight', 'cat', 'tree', 'bed', 'happy', 'go', '.DS_Store', 'dog', 'no', 'wow', 'nine', 'left', 'code', 'stop', 'three', '_background_noise_', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'off', 'four']


### Load Data

target list is ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unknown list is other
silence will be made from '_background_noise_'

Train data's sampling rate is 16000Hz, but for making lower computation cost, Resample to 8000hz

After training, test set also will resample to 8000Hz

In [150]:
dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()
print('Number of labels: ' + str(len(dirs[1:])))
print(dirs)

Number of labels: 31
['_background_noise_', 'bed', 'bird', 'cat', 'code', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [151]:
all_wav = []
unknown_wav = []
label_all = []
label_value = {}
target_list = ['code', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unknown_list = [d for d in dirs if d not in target_list and d != '_background_noise_' ]
print('target_list : ',end='')
print(target_list)
print('unknowns_list : ', end='')
print(unknown_list)
print('silence : _background_noise_')
i=0;
background = [f for f in os.listdir(join(train_audio_path, '_background_noise_')) if f.endswith('.wav')]
background_noise = []
for wav in background :
    samples, sample_rate = librosa.load(join(join(train_audio_path,'_background_noise_'),wav))
    samples = librosa.resample(samples, sample_rate, 8000)
    background_noise.append(samples)

for direct in dirs[1:]:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    label_value[direct] = i
    i = i + 1
    print(str(i)+":" +str(direct) + " ", end="")
    for wav in waves:
        samples, sample_rate = librosa.load(join(join(train_audio_path,direct),wav), sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if len(samples) != 8000 :
            continue

        if direct in unknown_list:
            unknown_wav.append(samples)
        else:
            label_all.append(direct)
            all_wav.append([samples, direct])

target_list : ['code', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unknowns_list : ['bed', 'bird', 'cat', 'dog', 'eight', 'five', 'four', 'happy', 'house', 'marvin', 'nine', 'one', 'seven', 'sheila', 'six', 'three', 'tree', 'two', 'wow', 'zero']
silence : _background_noise_


/var/folders/4v/sw011n6n73n99lwpjk6j8klm0000gp/T/ipykernel_9744/1332444937.py:17: FutureWarning:

Pass orig_sr=22050, target_sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error



1:bed 

/var/folders/4v/sw011n6n73n99lwpjk6j8klm0000gp/T/ipykernel_9744/1332444937.py:27: FutureWarning:

Pass orig_sr=16000, target_sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error



2:bird 3:cat 4:code 5:dog 6:down 7:eight 8:five 9:four 10:go 11:happy 12:house 13:left 14:marvin 15:nine 16:no 17:off 18:on 19:one 20:right 21:seven 22:sheila 23:six 24:stop 25:three 26:tree 27:two 28:up 29:wow 30:yes 31:zero 

split wav, label

In [152]:
wav_all = np.reshape(np.delete(all_wav,1,1),(len(all_wav)))
label_all = [i for i in np.delete(all_wav,0,1).tolist()]

/Users/user/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:5071: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



### Data Augmentation

For Data Augmentation. I will mix train wav, and same length(1 sec) noise(10%) from '_background_noise_'


In [153]:
#Random pick start point
def get_one_noise(noise_num = 0):
    selected_noise = background_noise[noise_num]
    start_idx = random.randint(0, len(selected_noise)- 1 - 8000)
    return selected_noise[start_idx:(start_idx + 8000)]

In [154]:
max_ratio = 0.1
noised_wav = []
augment = 1
delete_index = []
for i in range(augment):
    new_wav = []
    noise = get_one_noise(i)
    for i, s in enumerate(wav_all):
        if len(s) != 8000:
            print(delete_index, label_all[delete_index])
            delete_index.append(i)
            continue
        s = s + (max_ratio * noise)
        noised_wav.append(s)
np.delete(wav_all, delete_index)
np.delete(label_all, delete_index)

array(['down', 'down', 'down', ..., 'yes', 'yes', 'yes'], dtype='<U5')

In [155]:
wav_vals = np.array([x for x in wav_all])
label_vals = [x for x in label_all]
wav_vals.shape

(21312, 8000)

In [156]:
labels = copy.deepcopy(label_vals)
for _ in range(augment):
    label_vals = np.concatenate((label_vals, labels), axis = 0)
label_vals = label_vals.reshape(-1,1)

Random sampling from unknown wav data


In [157]:
#knowns audio random sampling
unknown = unknown_wav
np.random.shuffle(unknown_wav)
unknown = np.array(unknown)
unknown = unknown[:2000*(augment+1)]
unknown_label = np.array(['unknown' for _ in range(2000*(augment+1))])
unknown_label = unknown_label.reshape(2000*(augment+1),1)

May Some wav data has different length. So, Delete it

In [158]:
delete_index = []
for i,w in enumerate(unknown):
    if len(w) != 8000:
        delete_index.append(i)
unknown = np.delete(unknown, delete_index, axis=0)

Random sampling from '_background_noise_' 

Random pick background noise 


In [159]:
#silence audio
silence_wav = []
num_wav = (2000*(augment+1))//len(background_noise)
for i, _ in enumerate(background_noise):
    for _ in range((2000*(augment+1))//len(background_noise)):
        silence_wav.append(get_one_noise(i))
silence_wav = np.array(silence_wav)
silence_label = np.array(['silence' for _ in range(num_wav*len(background_noise))])
silence_label = silence_label.reshape(-1,1)
silence_wav.shape

(3996, 8000)

In [160]:
wav_vals    = np.reshape(wav_vals,    (-1, 8000))
noised_wav  = np.reshape(noised_wav,  (-1, 8000))
unknown       = np.reshape(unknown,   (-1, 8000))
silence_wav = np.reshape(silence_wav, (-1, 8000))

Check Dimensions

In [161]:
print(wav_vals.shape)
print(noised_wav.shape)
print(unknown.shape)
print(silence_wav.shape)

(21312, 8000)
(21312, 8000)
(4000, 8000)
(3996, 8000)


In [162]:
print(label_vals.shape)
print(unknown_label.shape)
print(silence_label.shape)

(42624, 1)
(4000, 1)
(3996, 1)


Concatenate wavs, labels 

In [163]:
wav_vals = np.concatenate((wav_vals, noised_wav), axis = 0)
wav_vals = np.concatenate((wav_vals, unknown), axis = 0)
wav_vals = np.concatenate((wav_vals, silence_wav), axis = 0)

In [164]:
label_vals = np.concatenate((label_vals, unknown_label), axis = 0)
label_vals = np.concatenate((label_vals, silence_label), axis = 0)

In [165]:
print(len(wav_vals))
print(len(label_vals))

50620
50620


In [166]:
train_wav, test_wav, train_label, test_label = train_test_split(wav_vals, label_vals, 
                                                                    test_size=0.2,
                                                                    random_state = 1993,
                                                                   shuffle=True)

In [167]:
# Parameters
lr = 0.001
generations = 20000
num_gens_to_wait = 250
batch_size = 512
drop_out_rate = 0.5
input_shape = (8000,1)

In [168]:
#For Conv1D add Channel
train_wav = train_wav.reshape(-1,8000,1)
test_wav = test_wav.reshape(-1,8000,1)

In [169]:
label_value = target_list
label_value.append('unknown')
label_value.append('silence')

In [170]:
new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

In [171]:
#Make Label data 'string' -> 'class num'
temp = []
for v in train_label:
    temp.append(label_value[v[0]])
train_label = np.array(temp)

temp = []
for v in test_label:
    temp.append(label_value[v[0]])
test_label = np.array(temp)

#Make Label data 'class num' -> 'One hot vector'
train_label = keras.utils.to_categorical(train_label, len(label_value))
test_label = keras.utils.to_categorical(test_label, len(label_value))

In [172]:
print('Train_Wav Demension : ' + str(np.shape(train_wav)))

Train_Wav Demension : (40496, 8000, 1)


In [173]:
print('Train_Label Demension : ' + str(np.shape(train_label)))

Train_Label Demension : (40496, 13)


In [174]:
print('Test_Wav Demension : ' + str(np.shape(test_wav)))

Test_Wav Demension : (10124, 8000, 1)


In [175]:
print('Test_Label Demension : ' + str(np.shape(test_label)))

Test_Label Demension : (10124, 13)


In [176]:
print('Number Of Labels : ' + str(len(label_value)))

Number Of Labels : 13


In [177]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 7990, 8)           96        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3995, 8)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 3995, 8)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 3989, 16)          912       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1994, 16)         0         
 1D)                                                         

In [178]:
#Conv1D Model
input_tensor = Input(shape=(input_shape))

x = layers.Conv1D(8, 11, padding='valid', activation='relu', strides=1)(input_tensor)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Conv1D(16, 7, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Conv1D(32, 5, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Conv1D(64, 5, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Conv1D(128, 3, padding='valid', activation='relu', strides=1)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(len(label_value), activation='softmax')(x)

model = tf.keras.Model(input_tensor, output_tensor)

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(lr = lr),
             metrics=['accuracy'])


/Users/user/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



### Train!

In [179]:
history = model.fit(train_wav, train_label, validation_data=[test_wav, test_label],
          batch_size=batch_size, 
          epochs=100,
          verbose=1)

Epoch 1/100


2022-11-12 10:18:37.670058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - ETA: 0s - loss: 2.4339 - accuracy: 0.1376

2022-11-12 10:19:06.222537: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 31s 318ms/step - loss: 2.4339 - accuracy: 0.1376 - val_loss: 2.3434 - val_accuracy: 0.1948
Epoch 2/100
80/80 [==============================] - 18s 224ms/step - loss: 2.2260 - accuracy: 0.2050 - val_loss: 2.2899 - val_accuracy: 0.1951
Epoch 3/100
80/80 [==============================] - 18s 221ms/step - loss: 2.0656 - accuracy: 0.2503 - val_loss: 2.0682 - val_accuracy: 0.2757
Epoch 4/100
80/80 [==============================] - 18s 219ms/step - loss: 1.8936 - accuracy: 0.3082 - val_loss: 1.8991 - val_accuracy: 0.3305
Epoch 5/100
80/80 [==============================] - 18s 221ms/step - loss: 1.7319 - accuracy: 0.3685 - val_loss: 1.8717 - val_accuracy: 0.3342
Epoch 6/100
80/80 [==============================] - 17s 217ms/step - loss: 1.6081 - accuracy: 0.4188 - val_loss: 1.7826 - val_accuracy: 0.3633
Epoch 7/100
80/80 [==============================] - 18s 223ms/step - loss: 1.5066 - accuracy: 0.4569 - val_loss: 1.7392 - val_accuracy: 0.3941
Epoc

In [180]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'acc'

In [181]:
model.save("../models")

INFO:tensorflow:Assets written to: ../models/assets


INFO:tensorflow:Assets written to: ../models/assets


In [182]:
model = keras.models.load_model('../models')

In [183]:
results = model.evaluate(test_wav, test_label, batch_size=128)


2022-11-12 10:55:28.608017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 2s 21ms/step - loss: 0.9133 - accuracy: 0.7211


In [184]:
def max_index(data):
    data = list(data)
    max_value = max(data)
    return data.index(max_value)


In [197]:
for index in range(10000, 15000):
    if(max_index(train_label[index]) == 1):
        print(max_index(model.predict(np.array((train_wav[index])).reshape((1, 8000, 1)))[0]))
        print(max_index(train_label[index]))
        sd.play(train_wav[index].flatten(), 8000)
        print("____")
        break

1/1 [==============================] - 0s 19ms/step
1
1
||PaMacCore (AUHAL)|| Warning on line 521: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Warning on line 441: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Error on line 1332: err='-10851', msg=Audio Unit: Invalid Property Value


PortAudioError: Error opening OutputStream: Internal PortAudio error [PaErrorCode -9986]

In [ ]:
print(set(label_vals.flatten()))

In [200]:
set(label_vals.flatten())

{'down',
 'go',
 'left',
 'no',
 'off',
 'on',
 'right',
 'silence',
 'stop',
 'unknown',
 'up',
 'yes'}

In [ ]:
,